# Asyncio 异步函数的使用

## TODO

In [1]:
import pandas as pd

In [ ]:

class AsyncHelper():
    def run(self,func,type=1):
        if type == 1:
            loop = asyncio.get_event_loop()
            loop.run_until_complete(func)
        else:

            asyncio.run(func)



### 编写一个协程函数

In [3]:
async def do_some_work(x):
    print('waiting',x)

In [4]:
async def do_some_work(x):
    print('waiting',x)
    return f'done after {x}'

In [5]:
coroutine = do_some_work(2)

### 将一个协程函数运行起来

In [6]:
import asyncio

loop = asyncio.get_event_loop()
task = loop.create_task(coroutine)
loop.run_until_complete(task)

waiting 2


'done after 2'

In [7]:
loop = asyncio.get_event_loop()
loop.run_until_complete(coroutine)#协程对象 -> task对象(Future)

RuntimeError: cannot reuse already awaited coroutine

### 获得协程函数的返回值

In [ ]:
def callback(future):
    print('Callback',future.result())

In [ ]:
task.add_done_callback(callback)

#### 更多的回调函数参数

In [25]:
def callback(t, future):
    print('Callback:', t, future.result())

task.add_done_callback(functools.partial(callback, 2))

NameError: name 'task' is not defined

#### 直接获取

In [ ]:
print('Task ret: ', task.result())

### 执行并发任务

In [ ]:
tasks = [
    asyncio.ensure_future(coroutine1),
    asyncio.ensure_future(coroutine2),
    asyncio.ensure_future(coroutine3)
]
loop.run_until_complete(asyncio.wait(tasks))

In [ ]:
for task in tasks:
    print('Task ret: ', task.result())

### 任务嵌套

In [ ]:
async def do_some_work(x):
    print('Waiting: ', x)

    await asyncio.sleep(x)
    return 'Done after {}s'.format(x)

In [ ]:

async def main():
    coroutine1 = do_some_work(1)
    coroutine2 = do_some_work(2)
    coroutine3 = do_some_work(4)

    tasks = [
        asyncio.ensure_future(coroutine1),
        asyncio.ensure_future(coroutine2),
        asyncio.ensure_future(coroutine3)
    ]

    dones, pendings = await asyncio.wait(tasks)

    for task in dones:
        print('Task ret: ', task.result())


In [ ]:
results = await asyncio.gather(*tasks)

for result in results:
    print('Task ret: ', result)

In [ ]:
async def main():
    coroutine1 = do_some_work(1)
    coroutine2 = do_some_work(2)
    coroutine3 = do_some_work(2)

    tasks = [
        asyncio.ensure_future(coroutine1),
        asyncio.ensure_future(coroutine2),
        asyncio.ensure_future(coroutine3)
    ]

    return await asyncio.gather(*tasks)

In [ ]:
async def main():
    coroutine1 = do_some_work(1)
    coroutine2 = do_some_work(2)
    coroutine3 = do_some_work(4)

    tasks = [
        asyncio.ensure_future(coroutine1),
        asyncio.ensure_future(coroutine2),
        asyncio.ensure_future(coroutine3)
    ]
    for task in asyncio.as_completed(tasks):
        result = await task
        print('Task ret: {}'.format(result))

start = now()

loop = asyncio.get_event_loop()
done = loop.run_until_complete(main())
print('TIME: ', now() - start)


### 协程停止

In [ ]:
import asyncio

import time

now = lambda: time.time()

async def do_some_work(x):
    print('Waiting: ', x)

    await asyncio.sleep(x)
    return 'Done after {}s'.format(x)

coroutine1 = do_some_work(1)
coroutine2 = do_some_work(2)
coroutine3 = do_some_work(2)

tasks = [
    asyncio.ensure_future(coroutine1),
    asyncio.ensure_future(coroutine2),
    asyncio.ensure_future(coroutine3)
]

start = now()

loop = asyncio.get_event_loop()
try:
    loop.run_until_complete(asyncio.wait(tasks))
except KeyboardInterrupt as e:
    print(asyncio.Task.all_tasks())
    for task in asyncio.Task.all_tasks():
        print(task.cancel())
    loop.stop()
    loop.run_forever()
finally:
    loop.close()

print('TIME: ', now() - start)


In [ ]:
import asyncio

import time

now = lambda: time.time()

async def do_some_work(x):
    print('Waiting: ', x)

    await asyncio.sleep(x)
    return 'Done after {}s'.format(x)

async def main():
    coroutine1 = do_some_work(1)
    coroutine2 = do_some_work(2)
    coroutine3 = do_some_work(2)

    tasks = [
        asyncio.ensure_future(coroutine1),
        asyncio.ensure_future(coroutine2),
        asyncio.ensure_future(coroutine3)
    ]
    done, pending = await asyncio.wait(tasks)
    for task in done:
        print('Task ret: ', task.result())

start = now()

loop = asyncio.get_event_loop()
task = asyncio.ensure_future(main())
try:
    loop.run_until_complete(task)
except KeyboardInterrupt as e:
    print(asyncio.Task.all_tasks())
    print(asyncio.gather(*asyncio.Task.all_tasks()).cancel())
    loop.stop()
    loop.run_forever()
finally:
    loop.close()

### 不同线程的事件循环

In [27]:
from threading import Thread

def start_loop(loop):
    asyncio.set_event_loop(loop)
    loop.run_forever()

def more_work(x):
    print('More work {}'.format(x))
    time.sleep(x)
    print('Finished more work {}'.format(x))

start = now()
new_loop = asyncio.new_event_loop()
t = Thread(target=start_loop, args=(new_loop,))
t.start()
print('TIME: {}'.format(time.time() - start))

new_loop.call_soon_threadsafe(more_work, 6)
new_loop.call_soon_threadsafe(more_work, 3)


TIME: 0.0011320114135742188


<Handle more_work(3) at /var/folders/sj/9gkwxdgs4h3ck3kytmdgk4240000gn/T/ipykernel_13458/1314391459.py:7>

More work 6
Finished more work 6
More work 3
Finished more work 3


In [28]:
def start_loop(loop):
    asyncio.set_event_loop(loop)
    loop.run_forever()

async def do_some_work(x):
    print('Waiting {}'.format(x))
    await asyncio.sleep(x)
    print('Done after {}s'.format(x))

def more_work(x):
    print('More work {}'.format(x))
    time.sleep(x)
    print('Finished more work {}'.format(x))

start = now()
new_loop = asyncio.new_event_loop()
t = Thread(target=start_loop, args=(new_loop,))
t.start()
print('TIME: {}'.format(time.time() - start))

asyncio.run_coroutine_threadsafe(do_some_work(6), new_loop)
asyncio.run_coroutine_threadsafe(do_some_work(4), new_loop)


TIME: 0.0007600784301757812


<Future at 0x7fe2606321a0 state=pending>

Waiting 6
Waiting 4
Done after 4s
Done after 6s


In [29]:
import asyncio

In [65]:
import queue

In [68]:
queue.Queue()

In [75]:
import queue
q = queue.Queue()  # 创建 Queue 队列
for i in range(3):
    q.put(i)  # 在队列中依次插入0、1、2元素
# for i in range(3):
    # print(q.get())  # 依次从队列中取出插入的元素，数据元素输出顺序为0、1、2


### 整合

In [ ]:
from threading import Thread

In [ ]:
def more_work(x):
    print('More work {}'.format(x))
    time.sleep(x)
    print('Finished more work {}'.format(x))


In [45]:

from threading import Thread

async def async_main(coroutines = []):
    """
    多任务同时执行 以最长的任务时间为限, 完毕后统一回复
    """
    tasks = [asyncio.ensure_future(cor_i) for cor_i in coroutines]
    if type == 'A':
        return await asyncio.gather(*tasks)
    elif type == 'B':
async def create_thread_async(coroutines = []):
    """
    多任务同时执行 创建一个线程去执行协程 而不会阻塞主线程
    """
    def start_loop(loop):
        asyncio.set_event_loop(loop)
        loop.run_forever()

    new_loop = asyncio.new_event_loop()
    t = Thread(target=start_loop, args=(new_loop,))
    t.start()
    for coroutine in coroutines:
        asyncio.run_coroutine_threadsafe(coroutine, new_loop)

        
loop = asyncio.get_event_loop()
task = asyncio.ensure_future(main())
try:
    loop.run_until_complete(task)
except KeyboardInterrupt as e:
    print(asyncio.Task.all_tasks())
    print(asyncio.gather(*asyncio.Task.all_tasks()).cancel())
    loop.stop()
    loop.run_forever()
finally:
    loop.close()

In [51]:
print(time.ctime())
a = await async_main([aa(1),aa(3)])
time.ctime()

Mon Mar  4 11:36:17 2024
111
111


'Mon Mar  4 11:36:20 2024'

In [ ]:

from threading import Thread
import asyncio

async def async_main(coroutines = []):
    """
    多任务同时执行 以最长的任务时间为限, 完毕后统一回复
    """
    tasks = [asyncio.ensure_future(cor_i) for cor_i in coroutines]

    return await asyncio.gather(*tasks)
async def create_thread_async(coroutines = []):
    """
    多任务同时执行 创建一个线程去执行协程 而不会阻塞主线程
    """
    def start_loop(loop):
        asyncio.set_event_loop(loop)
        loop.run_forever()

    new_loop = asyncio.new_event_loop()
    t = Thread(target=start_loop, args=(new_loop,))
    t.start()
    for coroutine in coroutines:
        asyncio.run_coroutine_threadsafe(coroutine, new_loop)


# loop = asyncio.get_event_loop()
# task = asyncio.ensure_future(main())
# try:
#     loop.run_until_complete(task)
# except KeyboardInterrupt as e:
#     print(asyncio.Task.all_tasks())
#     print(asyncio.gather(*asyncio.Task.all_tasks()).cancel())
#     loop.stop()
#     loop.run_forever()
# finally:
#     loop.close()